In [1]:
import sys, os
module_path = os.path.abspath(os.path.join('..'))
module_path = os.path.join(module_path, 'scripts')
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

## ePI Modules
from parse.rulebook.rulebook import styleRuleDict, styleFeatureKeyList
from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from  extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
%load_ext autoreload

%autoreload 2

C:\Users\vipsharm\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits \
            + string.ascii_lowercase) for _ in range(N))
    return str_



# Html Parsing Stage

In [100]:
module_path = os.path.abspath(os.path.join('..'))
module_path = os.path.join(module_path, 'data')
module_path = os.path.join(module_path, 'converted_to_html')

filenames = glob.glob(os.path.join(module_path, '*.html'))
filenames.extend(glob.glob(os.path.join(module_path, '*.htm')))

logger = loggerCreator('Parser_'+ getRandomString(1))
parserObj = parserExtractor(config, logger, styleRuleDict, styleFeatureKeyList)

for input_filename in filenames:
#     if(input_filename.find('emea-combined-h-2494-de')!=-1):
        output_filename = input_filename.replace('converted_to_html','outputJSON')
        output_filename = output_filename.replace('.html','.json')
        output_filename = output_filename.replace('.htm','.json')
        print(input_filename, output_filename)
        parserObj.createPIJsonFromHTML(input_filepath = input_filename,output_filepath = output_filename,img_base64_dict= parserObj.convertImgToBase64(input_filename))

F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\Abasaglar-h-2835-en.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\Abasaglar-h-2835-en.json
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\Abilify-h-471-e.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\Abilify-h-471-e.json
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\AbilifyMaintena-h-2755-e.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\AbilifyMaintena-h-2755-e.json
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\Adakveo-4874 EN PI.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\Adakveo-4874 EN PI.json
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\Adcetris-ema-combined-h-2455-en-IB-SOB-C2500.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\Adcetris-ema-combined-h-2

# Partition Stage

In [4]:
#qrdkeys = ['SmPC', 'ANNEX II', 'ANEXO III', 'B. PACKAGE LEAFLET']
## German
qrdkeys = ['ANEXO I','ANEXO II','ANEXO III','B. PROSPECTO']
path_json = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'outputJSON')
print(path_json)
partitionlogger = loggerCreator('Partition_'+ getRandomString(1))
partitioner = DocTypePartitioner(partitionlogger)
partitioner.partitionHtmls(qrdkeys, path_json)

2021-04-25 17:55:04,846 : Partition_m : Partitioning Json: emea-combined-h-2494-es.json


F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON
*************************** Texts with more than 2 characters**************************************


,Element,ID,Styles,Classes,Bold,Italics,Uppercased,Underlined,Indexed,IsListItem,HasBorder,IsPossibleHeading,IsHeadingType,Text,ParentId
0,"<div class=""WordSection1""> <p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p> <p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span ...",603f0744-ea60-46a5-a0d9-8674eac8d4dd,None,['WordSection1'],False,False,False,False,False,False,False,False,None,...,a8e75d1a-21a1-480c-a5a3-226c09194fd2
24,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES"">ANEXO I</span></b></p>",b8eaec4c-c8f6-4957-b2ad-c5f463a7d70f,text-align:center,['MsoNormal'],True,False,True,False,False,False,False,True,None,ANEXO I,603f0744-ea60-46a5-a0d9-8674eac8d4dd
26,"<p class=""TitleA""><span lang=""ES"">FICHA TÉCNICA O RESUMEN DE LAS CARACTERÍSTICAS DEL PRODUCTO</span></p>",f66371e4-593c-434d-a659-f50e59b40a46,None,['TitleA'],True,False,True,False,False,False,False,False,None,FICHA TÉCNICA O RESUMEN DE LAS CARACTERÍSTICAS DEL PRODUCTO,603f0744-ea60-46a5-a0d9-8674eac8d4dd
34,"<p class=""MsoNormal"" style=""margin-left:28.35pt;text-indent:-28.35pt;page-break-after: avoid""><b><span lang=""ES"">1. </span></b><b><span lang=""ES"">NOMBRE DEL MEDICAMENTO</span></b></p>",d1badb2a-282b-41b8-8285-e2ab478f4ae2,margin-left:28.35pt;text-indent:-28.35pt;page-break-after:\navoid,['MsoNormal'],True,False,True,False,True,False,False,True,L1,1. NOMBRE DEL MEDICAMENTO,603f0744-ea60-46a5-a0d9-8674eac8d4dd
36,"<p class=""MsoNormal""><span lang=""ES"">Kalydeco 75 mg comprimidos recubiertos con película</span></p>",1a2936a8-0002-49b7-a9eb-3f6be7e471e4,None,['MsoNormal'],False,False,False,False,False,False,False,False,None,Kalydeco 75 mg comprimidos recubiertos con película,603f0744-ea60-46a5-a0d9-8674eac8d4dd


2021-04-25 17:55:05,083 : Partition_m : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\emea-combined-h-2494-es_ANEXO I.json


*************************** Texts with more than 2 characters**************************************


...

,Element,ID,Styles,Classes,Bold,Italics,Uppercased,Underlined,Indexed,IsListItem,HasBorder,IsPossibleHeading,IsHeadingType,Text,ParentId
1339,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES"">ANEXO III</span></b></p>",4b18a3bf-32c1-4d32-97d8-f48b11d2192b,text-align:center,['MsoNormal'],True,False,True,False,False,False,False,True,L1,ANEXO III,603f0744-ea60-46a5-a0d9-8674eac8d4dd
1341,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES"">ETIQUETADO Y PROSPECTO</span></b></p>",cf602517-183b-414e-8966-0d82bae33ddb,text-align:center,['MsoNormal'],True,False,True,False,False,False,False,True,L1,ETIQUETADO Y PROSPECTO,603f0744-ea60-46a5-a0d9-8674eac8d4dd
1368,"<p class=""TitleA""><span lang=""ES"">A. ETIQUETADO</span></p>",5c3a5600-808f-4763-b7c1-cd35106dd963,None,['TitleA'],True,False,True,False,True,False,False,True,L2,A. ETIQUETADO,603f0744-ea60-46a5-a0d9-8674eac8d4dd
1373,"<div style=""border:solid windowtext 1.0pt;padding:1.0pt 4.0pt 1.0pt 4.0pt""> <p class=""MsoNormal"" style=""margin-left:28.35pt;text-indent:-28.35pt;border:none; padding:0in""><b><span lang=""ES"">INFORM...",4ba6ec50-2860-4d48-b745-6b81539d084b,border:solid windowtext 1.0pt;padding:1.0pt 4.0pt 1.0pt 4.0pt,None,False,False,False,False,False,False,True,False,None,,603f0744-ea60-46a5-a0d9-8674eac8d4dd
1374,"<p class=""MsoNormal"" style=""margin-left:28.35pt;text-indent:-28.35pt;border:none; padding:0in""><b><span lang=""ES"">INFORMACIÓN QUE DEBE FIGURAR EN EL EMBALAJE EXTERIOR</span></b></p>",98c696b3-420e-447e-9dbf-259055f1feda,margin-left:28.35pt;text-indent:-28.35pt;border:none;\npadding:0in,['MsoNormal'],True,False,True,False,False,False,False,True,L1,INFORMACIÓN QUE DEBE FIGURAR EN EL EMBALAJE EXTERIOR,4ba6ec50-2860-4d48-b745-6b81539d084b


2021-04-25 17:55:05,241 : Partition_m : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\emea-combined-h-2494-es_ANEXO III.json


*************************** Texts with more than 2 characters**************************************


,Element,ID,Styles,Classes,Bold,Italics,Uppercased,Underlined,Indexed,IsListItem,HasBorder,IsPossibleHeading,IsHeadingType,Text,ParentId
3223,"<p class=""TitleA""><span lang=""ES"">B. PROSPECTO</span></p>",e4fc9460-4597-4d81-84fd-b0ae37bfba7c,None,['TitleA'],True,False,True,False,True,False,False,True,None,B. PROSPECTO,603f0744-ea60-46a5-a0d9-8674eac8d4dd
3227,"<p align=""center"" class=""MsoNormal"" style=""text-align:center;page-break-after:avoid""><b><span lang=""ES"">Prospecto: información para el paciente</span></b></p>",7381a7b7-58a5-4a76-ba11-4fc20d6acb49,text-align:center;page-break-after:avoid,['MsoNormal'],True,False,False,False,False,False,False,True,L1,Prospecto: información para el paciente,603f0744-ea60-46a5-a0d9-8674eac8d4dd
3229,"<p align=""center"" class=""MsoNormal"" style=""text-align:center;page-break-after:avoid""><b><span lang=""ES"">Kalydeco 75 mg comprimidos recubiertos con película</span></b></p>",b12a6b4b-3458-45b9-aa77-428caba88d70,text-align:center;page-break-after:avoid,['MsoNormal'],True,False,False,False,False,False,False,True,L1,Kalydeco 75 mg comprimidos recubiertos con película,603f0744-ea60-46a5-a0d9-8674eac8d4dd
3230,"<p align=""center"" class=""MsoNormal"" style=""text-align:center;page-break-after:avoid""><b><span lang=""ES"">Kalydeco 150 mg comprimidos recubiertos con película</span></b></p>",fcdeb379-b7f4-4cb9-ab37-88a46538e719,text-align:center;page-break-after:avoid,['MsoNormal'],True,False,False,False,False,False,False,True,L1,Kalydeco 150 mg comprimidos recubiertos con película,603f0744-ea60-46a5-a0d9-8674eac8d4dd
3231,"<p align=""center"" class=""MsoNormal"" style=""text-align:center;page-break-after:avoid""><span lang=""ES"">ivacaftor</span></p>",d3382745-d001-4b02-986f-119ec76aa1bc,text-align:center;page-break-after:avoid,['MsoNormal'],False,False,False,False,False,False,False,False,None,ivacaftor,603f0744-ea60-46a5-a0d9-8674eac8d4dd


2021-04-25 17:55:05,282 : Partition_m : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\emea-combined-h-2494-es_ PROSPECTO.json


# Matching Stage

In [78]:
# Set variables for the specific section.

previousHeadingRowFound = None
procedureType = 'CAP'
languageCode = 'en'
documentType = 'SmPC'
stopWordlanguage = 'english'
docFilter = 'SmPC.json'
fileNameDoc = 'Kalydeco II-86-PI-clean_SmPC.json'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
fileNameMatchRuleBook = 'ruleDict.json'

stopWordFilterLen = 6 
topHeadingsConsidered = 4
bottomHeadingsConsidered = 6
isPackageLeaflet = False

In [79]:

# Set variables for the specific section.

previousHeadingRowFound = None
procedureType = 'CAP'
languageCode = 'en'
documentType = 'Package leaflet'
stopWordlanguage = 'english'
docFilter = 'LEAFLET.json'
fileNameDoc = 'Abilify-h-471-e_ PACKAGE LEAFLET.json'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
fileNameMatchRuleBook = 'ruleDict.json'

stopWordFilterLen = 100
topHeadingsConsidered = 5
bottomHeadingsConsidered = 10
isPackageLeaflet = True

In [80]:

# Set variables for the specific section.

procedureType = 'CAP'
languageCode = 'es'
documentNumber = 0
stopWordlanguage = 'spanish'
fileNameDoc = 'emea-combined-h-2494-es_ANEXO I.json'
fileNameQrd = 'qrd_canonical_mode_CAP_NAP.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'
stopWordFilterLen = 6 
topHeadingsConsidered = 4
bottomHeadingsConsidered = 6
isPackageLeaflet = False


###
#ANGABEN AUF DER ZWISCHENVERPACKUNG
###



In [81]:
matchDocObj = MatchDocument(procedureType,
                 languageCode,
                 documentNumber,
                 fileNameDoc,
                 fileNameQrd,
                 fileNameMatchRuleBook,
                 fileNameDocumentTypeNames,
                 topHeadingsConsidered,
                 bottomHeadingsConsidered,
                 stopWordFilterLen,
                 stopWordlanguage,
                 isPackageLeaflet,'Kalydeco')

File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\emea-combined-h-2494-es_ANEXO I.json
--------------------------------------------


In [82]:
    df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

----------------------------------
RemovedByStyle  ||    ||  FICHA TÉCNICA O RESUMEN DE LAS CARACTERÍSTICAS DEL PRODUCTO  ||  FICHA TÉCNICA O RESUMEN DE LAS CARACTERÍSTICAS DEL PRODUCTO
----------------------------------
True  ||  <=4|4.0|(98, 96, 98)|0.99|  ||  1.       NOMBRE DEL MEDICAMENTO  ||  1 NOMBRE DEL MEDICAMENTO
True  ||  <=7|2.44|(99, 98, 99)|1.0|  ||  2.       COMPOSICIÓN CUALITATIVA Y CUANTITATIVA  ||  2 COMPOSICIÓN CUALITATIVA Y CUANTITATIVA


Heading Not Found None 


True  ||  <=4|10.0|(95, 90, 95)|0.99|  ||  Excipiente con efecto conocido  ||  Excipiente(s) con efecto conocido


Heading Not Found None 


True  ||  <=4|10.0|(95, 90, 95)|0.99|  ||  Excipiente con efecto conocido  ||  Excipiente(s) con efecto conocido


Heading Not Found None 


True  ||  <=4|4.76|(98, 95, 97)|0.99|  ||  3.       FORMA FARMACÉUTICA  ||  3 FORMA FARMACÉUTICA

OriginalCheck
('<=4|76.47|(24, 25, 97)|0.5|', '4.       Datos clínicos', '4 DATOS CLÍNICOS')

True  ||  checkLowerCase|5.88|(97, 94



Mismatch




Mismatch




Mismatch




Mismatch




Mismatch


True  ||    ||  Población pediátrica  ||  Población pediátrica
True  ||  <=4|12.5|(88, 88, 90)|0.95|  ||  5.2     Propiedades farmacocinéticas  ||  5.1 Propiedades farmacodinámicas


Heading Not Found None 




Heading Not Found None 




Heading Not Found None 




Heading Not Found None 




Heading Not Found None 




KeyboardInterrupt: 

In [65]:
df

,Element,ID,Styles,Classes,Bold,Italics,Uppercased,Underlined,Indexed,IsListItem,HasBorder,IsPossibleHeading,IsHeadingType,Text,ParentId,StringLength
0,"<div class=""WordSection1""> <p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p> <p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span ...",af72c67e-e145-4951-a2ae-9d6e209e11df,None,['WordSection1'],False,False,False,False,False,False,False,False,None,...,f8cea01e-5a38-4761-a231-51d257e0ad34,0
1,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",c147a932-fe2e-4586-ad7a-1d94722a6bbd,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
2,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",354c4ef9-6998-4604-a3de-d6c2b54ab5bd,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
3,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",84207554-4f1a-44fe-a4af-2faa4181e82f,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
4,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",666eb08a-4aa5-4ea8-a12e-2e1b48cdc1d9,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
5,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",41ca5b92-4733-4f9a-ac58-e7182fb0d166,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
6,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",9e6429ad-0e94-4062-b75f-b03eb212e2d1,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
7,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",2c762f89-12bd-4ba8-b435-7dd0ab82b1c1,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
8,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",898da31c-e821-4aa6-998f-c25d22b10de7,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0
9,"<p align=""center"" class=""MsoNormal"" style=""text-align:center""><b><span lang=""ES""> </span></b></p>",d84210a2-e953-4ced-916c-00fb0d485c9a,text-align:center,['MsoNormal'],False,False,False,False,False,False,False,False,None,,af72c67e-e145-4951-a2ae-9d6e209e11df,0


# Content Extraction Stage

In [83]:
extractContentlogger = loggerCreator('ExtractContentBetween_'+ getRandomString(1))
extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll)
dfExtractedHierRR = extractorObj.extractContentBetweenHeadings('Abilify-h-471-e_ PACKAGE LEAFLET.json')

2021-04-22 11:47:46,065 : ExtractContentBetween_y : Cleaning Match Results
2021-04-22 11:47:46,070 : ExtractContentBetween_y : Finished Cleaning Match Results
2021-04-22 11:47:46,098 : ExtractContentBetween_y : Extracting Content Between Headings
2021-04-22 11:47:46,121 : ExtractContentBetween_y : Finished Extracting Content Between Headings


File being processed: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\data\partitionedJSONs\Abilify-h-471-e_ PACKAGE LEAFLET.json
--------------------------------------------


,index,id,Procedure type,Display code,Name,parent_id,htmlText,htmlIndex,htmlId,SubSectionIndex,doc_parent_id,Text,Html_betw
0,786,23001,CAP,NaN,PACKAGE LEAFLET,NaN,B. PACKAGE LEAFLET,24,ba7f0011-777c-4c87-ab14-945b7f219145,0,,\nB. PACKAGE LEAFLET\n\n \n\nPackage leaflet: Information for the user\n \nABILIFY 5 mg tablets\nABILIFY 10 mg tablets\nABILIFY 15 mg tablets\nABILIFY 30 mg tablets\n \naripiprazole\n \nRead all o...,"<p class=""TitleA""><span lang=""EN-GB"" style=""color:black"">B. PACKAGE LEAFLET</span></p><b><span lang=""EN-GB"" style='font-size:11.0pt;font-family:""Times New Roman"",serif; color:black'><br clear=""all..."
1,788,23003,CAP,NaN,What is in this leaflet,23001,What is in this leaflet,43,6399df46-b26b-41ef-8faa-1e468be43bd7,0,23001,\nWhat is in this leaflet\n1. What ABILIFY is and what it is used for\n2. What you need to know before you take ABILIFY\n3. How to take ABILIFY\n4. Possible side effects\n5 ...,"<p class=""MsoNormal""><b><span lang=""EN-GB"" style=""color:black"">What is in this leaflet</span></b></p><p class=""EMEABodyText"" style=""margin-left:28.35pt;text-indent:-28.35pt""><span lang=""EN-GB"" sty..."
2,789,23004,CAP,1,What X is and what it is used for,23001,1. What ABILIFY is and what it is used for,52,10306e47-6cf5-4529-b240-c8c69af4c4c7,0,23001,\n1. What ABILIFY is and what it is used for\n \nABILIFY contains the active substance aripiprazole and belong to a group of medicines called antipsychotics. It is used to treat adults and a...,"<p class=""MsoNormal"" style=""margin-left:28.35pt;text-indent:-28.35pt""><b><span lang=""EN-GB"" style=""color:black"">1. What ABILIFY is and what it is used for</span></b></p><p class=""EMEABodyTex..."
3,790,23005,CAP,2,What you need to know before you <take> <use> X,23001,2. What you need to know before you take ABILIFY,59,1fe30967-1779-470d-95e2-9c90a04c60c4,0,23001,\n2. What you need to know before you take ABILIFY\n,"<p class=""MsoNormal"" style=""margin-left:28.35pt;text-indent:-28.35pt""><b><span lang=""EN-GB"" style=""color:black"">2. What you need to know before you take ABILIFY</span></b></p><p class=""EMEAB..."
4,791,23006,CAP,NaN,Do not <take> <use> X,23005,Do not take ABILIFY,61,b62e92fa-848f-4e9e-b9cb-b3b7fb89afdf,0,23005,\nDo not take ABILIFY\n• if you are allergic to aripiprazole or any of the other ingredients of this medicine (listed in section 6).\n,"<p class=""MsoNormal""><b><span lang=""EN-GB"" style=""color:black"">Do not take ABILIFY</span></b></p><p class=""EMEABodyTextIndent""><span lang=""EN-GB"" style=""color:black"">• if you are allergic t..."
5,792,23007,CAP,NaN,Warnings and precautions,23005,Warnings and precautions,64,20686598-1bda-4358-a32d-52fc5ce73748,0,23005,\nWarnings and precautions\nTalk to your doctor before taking ABILIFY.\n \nSuicidal thoughts and behaviours have been reported during aripiprazole treatment. Tell your doctor immediately if you ar...,"<p class=""MsoNormal""><b><span lang=""EN-GB"" style=""color:black"">Warnings and precautions</span></b></p><p class=""EMEABodyText""><span lang=""EN-GB"" style=""color:black"">Talk to your doctor before taki..."
6,793,23008,CAP,NaN,Children <and adolescents>,23005,Children and adolescents,90,2ef4ce01-dcb9-4ef2-bf32-9ce8368cfae7,0,23005,\nChildren and adolescents\nDo not use this medicine in children and adolescents under 13 years of age. It is not known if it is safe and effective in these patients.\n,"<p class=""MsoNormal""><b><span lang=""EN-GB"" style=""color:black"">Children and adolescents</span></b></p><p class=""MsoNormal""><span lang=""EN-GB"" style=""color:black"">Do not use this medicine in childr..."
7,794,23009,CAP,NaN,Other medicines and X,23005,Other medicines and ABILIFY,93,64c7ac18-2ac4-43bf-bd98-1338a58e3276,0,23005,"\nOther medicines and ABILIFY\nTell your doctor or pharmacist if you are taking, have recently taken or might take any other medicines, including medicines obtained without a prescription.\n \nBlo...","<p class=""MsoNormal""><b><span lang=""EN

# XML Generation Stage

In [84]:
xmlLogger = loggerCreator('XmlGeneration_'+ getRandomString(1))
fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger)
fhirXmlGeneratorObj.generateXml(dfExtractedHierRR,  'Abilify-h-471-e_ PACKAGE LEAFLET.xml')

2021-04-22 11:47:56,146 : XmlGeneration_4 : Initiating XML Generation
2021-04-22 11:47:56,415 : XmlGeneration_4 : Writing to File:Abilify-h-471-e_ PACKAGE LEAFLET.xml


In [75]:
convertCollectionToDataFrame(coll)

,index,id,Procedure type,Display code,Name,parent_id,htmlText,htmlIndex,htmlId,SubSectionIndex,doc_parent_id
0,786,23001,CAP,NaN,PACKAGE LEAFLET,NaN,B. PACKAGE LEAFLET,24,ba7f0011-777c-4c87-ab14-945b7f219145,0,
1,788,23003,CAP,NaN,What is in this leaflet,23001,What is in this leaflet,43,6399df46-b26b-41ef-8faa-1e468be43bd7,0,23001
2,789,23004,CAP,1,What X is and what it is used for,23001,1. What ABILIFY is and what it is used for,52,10306e47-6cf5-4529-b240-c8c69af4c4c7,0,23001
3,790,23005,CAP,2,What you need to know before you <take> <use> X,23001,2. What you need to know before you take ABILIFY,59,1fe30967-1779-470d-95e2-9c90a04c60c4,0,23001
4,791,23006,CAP,NaN,Do not <take> <use> X,23005,Do not take ABILIFY,61,b62e92fa-848f-4e9e-b9cb-b3b7fb89afdf,0,23005
5,792,23007,CAP,NaN,Warnings and precautions,23005,Warnings and precautions,64,20686598-1bda-4358-a32d-52fc5ce73748,0,23005
6,793,23008,CAP,NaN,Children <and adolescents>,23005,Children and adolescents,90,2ef4ce01-dcb9-4ef2-bf32-9ce8368cfae7,0,23005
7,794,23009,CAP,NaN,Other medicines and X,23005,Other medicines and ABILIFY,93,64c7ac18-2ac4-43bf-bd98-1338a58e3276,0,23005
8,795,23010,CAP,NaN,"X with <food> <and> <,> <drink> <and> <alcohol>",23005,"ABILIFY with food, drink and alcohol",121,bf52352e-edd8-4b45-ba51-e791cc8fdcd4,0,23005
9,796,23011,CAP,NaN,"Pregnancy <and> <,> breast-feeding <and fertility>",23005,"Pregnancy, breast-feeding and fertility",125,7294f0d4-67e1-40ff-bfc6-4effd7beb960,0,23005


In [326]:
documentAnnotationObj = DocumentAnnotation('Kalydeco II-86-PI-clean_SmPC.json','c270d6ccaf9e47e9b20b322e2383c4ba','https://spor-uat.azure-api.net/pms/api/v2/',df,coll)

In [327]:
documentAnnotationObj.processRegulatedAuthorizationForDoc()

 ['EU/1/12/782/001', 'EU/1/12/782/002', 'EU/1/12/782/005', 'EU/1/12/782/003', 'EU/1/12/782/004', 'EU/1/12/782/006']
EU/1/12/782/001


MissingKeyValuePair: Missing Key 'entry' in the regulated authorization API output

In [328]:
documentAnnotationObj.processRegulatedAuthorizationForDoc(['EU/3/00/001','EU/1/97/039/003'])

EU/3/00/001
Skipping entry due to incorrect code 220000000062
No Regulated Authorization find with code 220000000061
Skipping entry due to incorrect code 220000000062
No Regulated Authorization find with code 220000000061
Skipping entry due to incorrect code 220000000062
No Regulated Authorization find with code 220000000061
EU/1/97/039/003
Found entry with code 220000000061
['0', '600000034241']
Cystagon 150 mg - Capsule, hard


[('0', '600000034241', 'Cystagon 150 mg - Capsule, hard')]